# Derived Tables

> Udviklet af Thomas Lange & Mick Ahlmann Brun

Mere info: [https://github.com/M1ckB/T-SQL](https://github.com/M1ckB/T-SQL)

Version 1.0 2023-02-02

Laboratoriet kræver:

- En understøttet version af SQL Server
- En Stack Overflow database: [Brent Ozar](https://www.BrentOzar.com/go/querystack) (medium)

Læs mere om derived tables i Microsofts T-SQL reference:

- [https://learn.microsoft.com/en-us/sql/relational-databases/performance/subqueries?view=sql-server-ver16](https://learn.microsoft.com/en-us/sql/relational-databases/performance/subqueries?view=sql-server-ver16)


## Indholdsfortegnelse

- [Introduktion](#Introduktion)
- [Afledte tabeller](#Afledte-tabeller)
  - [Referencer og nesting](#Referencer-og-nesting)
- [Hovedpointer](#Hovedpointer)

## Introduktion

En *derived table*, eller en afledt tabel, er strengt taget en (tabel) subquery, men er udskilt herfra fordi det samtidig hører under kategorien af *table expressions*, eller tabeludtryk.

Et tabeludtryk er en query som repræsenterer en gyldig relationel tabel. Tabeludtryk kan i høj grad bruges lige som man bruger tabeller. T-SQL understøtter, ud over derived tables, følgende tabeludtryk: [common table expression (CTEs)](CTE.ipynb), [views](Views.ipynb) og inline table-valued functions.

Tabeludtryk er logiske eller virtuelle, dvs. de bliver ikke materialiseret nogle steder. Når man forespørger et tabeludtryk, så vil elementer fra den *indre* forespørgsel udfoldes således der forespørges direkte på de underliggende objekter for den samlede forespørgsel.

Use cases:

- Muliggør delvise beregninger
- Simplificerer kode ved modulopbygning
- Overkommer begrænsninger i T-SQL, særligt i forhold til de logiske faser og all-at-once logik


## Afledte tabeller

En afledt tabel defineres i `FROM`-delsætningen af en *ydre* forespørgsel. Den eksisterer kun i scopet af den *ydre forespørgsel*.

Forespørgslen som udgør den afledte tabel angives i parenteser og gives et alias.

```sql
SELECT
    col
FROM (
    SELECT
        col
    FROM tableA
) AS a
```

Den afledte tabel skal opfylde følgende kriterier for en relation, herunder:

- Orden er ikke garanteret
- Kolonner skal have navne
- Kolonnenavne skal være unikke


In [ ]:
/* Eksempel på en afledt tabel */

CREATE TABLE #TableA (
    Id int NOT NULL,
    Col nvarchar(3) NULL
)

INSERT INTO #TableA (Id, Col)
VALUES
(1, 'No'), (2, 'Yes'), (3, 'No');

SELECT * FROM #TableA;

SELECT
    *
FROM (
    SELECT
        Id,
        Col
    FROM #TableA
    WHERE Col = 'No'
) AS a;

DROP TABLE #TableA;

### Referencer og nesting

Når du benytter afledte tabeller, så kan du i den *ydre* forespørgsel referere kolonner fra den *indre* forespørgsel. Dette gør det muligt at overkomme problemer med at beregninger som laves i `SELECT`-fasen ikke kan refereres i faserne før, fx `WHERE`.

Derudover så kan afledte tabeller nestes, dvs. en afledt tabel kan godt selv referere en afledt tabel. Denne praksis er ikke anbefalelsesværdig da koden som regel bliver kompliceret og ulæselig.

Til sidst kan det nævnes at afledte tabeller ikke kan genbruges.


In [ ]:
/* Eksempel på referencer og nesting */

CREATE TABLE #TableA (
    Id int NOT NULL,
    Dat date NULL
)

INSERT INTO #TableA (Id, Dat)
VALUES
(1, '20220102'), (2, '20220709'), (3, '20230201');

SELECT * FROM #TableA;

SELECT
    *
FROM (
    SELECT
        Id,
        YEAR(Dat) AS Yr
    FROM #TableA
) AS a
WHERE Yr = 2022;

SELECT
    *
FROM (
    SELECT Yr, COUNT(*) AS Count
    FROM (
        SELECT Id, YEAR(Dat) AS Yr
        FROM #TableA
    ) AS a1
    WHERE Yr = 2022
    GROUP BY Yr
) AS a2
WHERE Count > 1;

DROP TABLE #TableA;

### *Tid til opgaver...*

Lav opgave 1.1, 1.2, 2.1 og 2.2 i [opgavehæftet](Derived-tables.sql).

## Hovedpointer

- En *derived table*, eller en afledt tabel, er en *tabel subquery*
- En afledt tabel skal repræsentere en gyldig relationel tabel
- En afledt tabel defineres i `FROM`-delsætningen og dens scope er den *ydre* forespørgsel
- Den ydre forespørgsel kan referere kolonner fra den indre forespørgsel (den afledte tabel)
- Man kan *neste* afledte tabeller
- En afledt tabel kan ikke bruges flere gange i en forespørgsel


## Licens

Creative Commons Attribution-ShareAlike 4.0 International (CC BY-SA 4.0)

Mere info: [https://creativecommons.org/licenses/by-sa/4.0/](https://creativecommons.org/licenses/by-sa/4.0/)

Du kan frit:

- Dele: kopiere og distribuere materialet via ethvert medium og i ethvert format
- Tilpasse: remixe, redigere og bygge på materialet til ethvert formål, selv erhvervsmæssigt

Under følgende betingelser:

- Kreditering: Du skal kreditere, dele et link til licensen og indikere om der er lavet ændringer.
- Del på samme vilkår: Hvis du remixer, redigerer eller bygger på materialet, så skal dine bidrag
  distribueres under samme licens som den originale.
